In [ ]:
# Scrape LinkedIn companies based on user inputted search query and update into Company Web Scrape Google Sheets 
# Script for LinkedIn Company Mining Feature

#import libraries
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time, sys, requests, random

# import and authorize gspread  
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
google_key_file = '/Users/zacharywong/Documents/ServiceAccountKey-Secret/pelagic-tracker-338302-eaf0e0e671cb.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)
#Global variables
query = ''
queries = []
queriesSplit = []
companies = []
employees = []
locations = []
industries = []
employeeLinks = []
results = []
numberofPages = 0
cellLocationQuery = 'C2'
cellLocationPages = 'B2'
sheetIndexRead = 0
sheetIndexWrite = 2
columnName = 'Company Name (Leave Blank if Using LinkedIn Company Mining)'
columnEmployee = 'Number of Employees'
columnLocation = 'Company Location'
columnIndustry = 'Company Industry'
password = ''
email = 'zach.jl.wong@gmail.com'
drivers = []
namesList = []
maxThreads = 4

#paths/urls
spreadsheet_id = '1vFXonFCyUlEKa1f0s5tvHCKeTek_sAv7rUPYfYss0Qo'
loginurl = 'https://www.linkedin.com/uas/login'
driver_path = '/Users/zacharywong/Documents/Work/Portfolio/DigitalHealthWebscrape/chromedriver'
pathtoLinkedInFile = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Misc/Companies/Company Web Scrape Tool - Company Names.csv'
pathtoPassword = '/Users/zacharywong/Documents/LinkedIn/LinkedIn-Password-Secret.txt'

#waitTimes
waitLinkedInTime = 10
waitCaptcha = 25
waitSearch= 1
waitLogin = 0.8
waitEmployeeCount=2.5


# xPaths, classes, and javascripts
employeeCountXPATH = "//span[@class='link-without-visited-state t-bold t-black--light']"
employeeCount1XPATH = "//span[@class='org-top-card-secondary-content__see-all t-normal t-black--light link-without-visited-state link-without-hover-state']"
companyIndustryXPATH = "//div[@class='org-top-card-summary-info-list__info-item']"
companyLocationXPATH = "//div[@class='inline-block']"
locationFilterXPATH = "//button[text()='Locations']"
northAmericaXPATH = '//*[@id="companyHqGeo-102221843"]'
submitXPATH = '//*[@id="ember328"]'
searchBarXPath = '//*[@id="global-nav-typeahead"]/input'
companyFilterXPath = '//*[@id="search-reusables__filters-bar"]/ul/li[2]/button'
scrollDownScript = "window.scrollTo(0,document.body.scrollHeight)"
pageButtonClass = "button[type='button']"
companyTitleClass = 'app-aware-link'
companyTitleAttr = 'a'
employeeAttr = 'a'
employeeClass = 'app-aware-link'


# helper function to read in value from spreadsheet
def readinValue(cellLocation, sheetIndexRead):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndexRead)
    try:
        value = worksheet.acell(cellLocation).value
    except:
        wait()
    return value

def readQueries():
    global queries
    row = 2;
    sheetIndex = 0
    isDone = False
    cellLocationColumn = 'C'
    while (isDone == False):
        cellLocationKeyWords = cellLocationColumn + str(row)
        try:
            keyword = readinValue(cellLocationKeyWords, sheetIndex)
            queries.append(keyword)
            if(keyword == None):
                isDone = True
                break
            else:
                row += 1 
        except:
            wait()
    queries = queries[0:len(queries)-1]
    print('Queries: ', str(queries))
    
# read in query from spreadsheet
def readinQuery(queryIndex):
    global query
    query = queries[queryIndex]
    print('currentQuery: ' + query)
    
    
# read in number of pages from spreadsheet
def readinPages():
    global numberofPages
    numberofPages = int(readinValue(cellLocationPages, sheetIndexRead))
    print('numberofPages: ', numberofPages)
    
    
    
# login to linkedin 
def login(driver):
    driver.get(loginurl)
    emailInput = driver.find_element(By.ID, 'username')
    emailInput.send_keys(email)
    time.sleep(waitLogin)
    passwordInput = driver.find_element(By.ID, 'password')
    with open (pathtoPassword, 'r') as file:
        global password
        password = file.read()
    passwordInput.send_keys(password)
    time.sleep(waitLogin)
    passwordInput.send_keys(Keys.RETURN)
    
    
# search query on LinkedIn
def searchLinkedIn(driver):
    #search query
    try:
        searchBar = driver.find_element(By.XPATH, searchBarXPath)
        searchBar.send_keys(query)
        searchBar.send_keys(Keys.RETURN)
        time.sleep(1)
    except: 
        time.sleep(waitCaptcha)
        searchBar = driver.find_element(By.XPATH, searchBarXPath)
        searchBar.send_keys(query)
        searchBar.send_keys(Keys.RETURN)
        time.sleep(1)
        
        
# Click Company Filter option on LinkedIn
def companyFilter(driver):
    WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, companyFilterXPath)))
    companyButtons = driver.find_elements_by_css_selector(pageButtonClass)
    for button in companyButtons: 
        buttonText = button.text
        if (buttonText == 'Companies'): 
            button.click()
            return;
# Click onto next page 


# Click onto next page 
def nextPage(currentPage, driver):
    nextPageSuccess = False; 
    driver.execute_script(scrollDownScript)
    time.sleep(1)
    #WebDriverWait(driver, waitLinkedInTime).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ember329"]/button')))
    pages = driver.find_elements_by_css_selector(pageButtonClass)
    for page in pages:
        #time.sleep(1)
        buttonNumber = page.text
        #print(buttonNumber) 
        if (buttonNumber == str(currentPage)):
            #print('hello')
            page.click()
            nextPageSuccess = True; 
            break
    if (nextPageSuccess == False):
        for page in pages:
            buttonNumber = page.text
            if (buttonNumber == '…'):
                print("click more option")
                page.click()
                return 
            
            
# add companies into a pandas dataframe 
def addCompanies(driver):
    global companies
    currentPage = 1
    while (currentPage <= numberofPages):
        time.sleep(2)
        soup = bs(driver.page_source, 'html.parser')
        companySearch = soup.find_all(companyTitleAttr, class_ = companyTitleClass)
        for company in companySearch:
            companyName = company.text
            if '\n\n\n' not in companyName and companyName not in companies:
                companies.append(companyName.replace('\n', ''))
        currentPage += 1
        nextPage(currentPage, driver)
    return companies

def addLocation(driver):
    pattern = '[0-9]+'
    try:
        WebDriverWait(driver, waitEmployeeCount).until(EC.presence_of_element_located((By.XPATH, companyLocationXPATH)))
        location = driver.find_element(By.XPATH, companyLocationXPATH)
    except:
        print("cannot find location")
        global locations
        locations.append(None)
        return
    location = location.text
    location = re.split(pattern, location)
    location = location[0]
    print(location)
    locations.append(location)
    return
    
def addIndustry(driver):
    try:
        WebDriverWait(driver, waitEmployeeCount).until(EC.presence_of_element_located((By.XPATH, companyIndustryXPATH)))
        industry = driver.find_element(By.XPATH, companyIndustryXPATH)
        print(industry.text)
        global industries
        industries.append(industry.text)
    except:
        print("cannot find industry")
        industries.append(None)
    return
    
def addSecondaryInfo():
    print('employee links: ', employeeLinks)
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    login(driver)
    time.sleep(waitCaptcha)
    for link in employeeLinks:
        isDone = False
        driver.get(link)
        try:
            WebDriverWait(driver, waitEmployeeCount).until(EC.presence_of_element_located((By.XPATH, employeeCountXPATH)))
            employee = driver.find_element(By.XPATH, employeeCountXPATH)
            print(employee.text)
        except: 
            print("cannot find regular employee")
            try:
                WebDriverWait(driver, waitEmployeeCount).until(EC.presence_of_element_located((By.XPATH, employeeCount1XPATH)))
                employee = driver.find_element(By.XPATH, employeeCount1XPATH)
                print(employee.text)
            except:
                print("cannot find employee location")
                global employees 
                employees.append(None)
                isDone = True
       
        addIndustry(driver)
        addLocation(driver)      
        
        if isDone == False: 
            employeeCount = employee.text.split(' ')
            #primary case
            if (employeeCount[1] == 'all'):
                employeeCount = employeeCount[2]
            #secondary case
            elif(employeeCount[1] == 'employees'):
                employeeCount = employeeCount[0]
            #1 employee for primary case 
            else:
                employeeCount = employeeCount[1]
            print("count: " + employeeCount + ' employeeLink: ' + str(link))
            employees.append(employeeCount)
        else: 
            continue
    print('employeeCounts: ' + str(employees) + str(len(employees)) + '\n' )
    print('locations: ' + str(locations) + str(len(locations)) + '\n')
    print('industries: ' + str(industries) + str(len(industries)) + '\n')
    time.sleep(0.4)


def makeDF():
    tuples = list(zip(companies, employees, locations, industries))
    df = pd.DataFrame(tuples, columns = [columnName, columnEmployee, columnLocation, columnIndustry])
    return df 

# export final list of companies to CSV 
def exportCSV(df):
    df.to_csv(pathtoLinkedInFile)


# Update spreadsheet with list of companies
def updateSpreadSheet(df, sheetIndexWrite):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndexWrite)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    
    
# run all functions to scrape LinkedIn companies  
def runLinkedIn(queryIndex):
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    readinQuery(queryIndex)
    readinPages()
    login(driver)
    searchLinkedIn(driver)
    companyFilter(driver)
    companies = addCompanies(driver)
    
# run program 
startTime = time.time()
readQueries()
print('Queries: ', queries)
queryIndex = 0
while queryIndex < len(queries):
    runLinkedIn(queryIndex)
    print(companies)
    queryIndex += 1
addSecondaryInfo()
df = exportCSV(companies)
updateSpreadSheet(df, sheetIndexWrite)
endTime = time.time()
elapsedTime = endTime - startTime
print('Time Elapsed: ', elapsedTime)
df